In [2]:
import json
with open("bikes.json") as f:
    dataset = json.load(f)
    
    dataset.keys()

type(dataset["network"])
dataset["network"].keys()
type(dataset["network"]["stations"])
dataset["network"]["stations"][0]

{'empty_slots': 3,
 'extra': {'number': 1,
  'reviews': 72,
  'score': 4.0,
  'status': 'offline',
  'uid': '537'},
 'free_bikes': 5,
 'id': 'f01c3eb11d001cdeb048c5153d2312ae',
 'latitude': 45.04440354326403,
 'longitude': 7.617623805999756,
 'name': '01. Gerbido',
 'timestamp': '2024-03-27T04:03:56.201000Z'}

In [3]:
active_stations = [ station for station in dataset["network"]["stations"] if station["extra"]["status"] == "online" ]
print("Number of active stations", len(active_stations))
bikes_avail = sum([ station["free_bikes"] for station in dataset["network"]["stations"]])
free_docks = sum([ station["empty_slots"] for station in dataset["network"]["stations"]])
print("Bikes available", bikes_avail)
print("Free docks", free_docks)

Number of active stations 1
Bikes available 95
Free docks 62


In [4]:
from math import cos, acos, sin

def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))
# distance, in meters
distance_coords(45.074512, 7.694419, 45.075309, 7.695369)

115.9682872491932

In [5]:
def distance_from_point(dataset, lat, lng):
    closest = (None, None)
    for station in dataset["network"]["stations"]:
        closest_station, closest_distance = closest
        current_distance = distance_coords(lat, lng, station["latitude"], station["longitude"])
        # if closest_distance is None, then we are at the first
        # loop execution where the station has available bikes.
        # In that case, we save the current station as the 
        # closest one (as we do not have any other stations available).
        # From the next cycle on, to update `closest`, we need
        # the station to actually be closer than the already saved one.
        if station["free_bikes"] > 0 and (closest_distance is None or current_distance < closest_distance):
            closest = (station, current_distance)
    return closest

station, distance = distance_from_point(dataset, 45.074512, 7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")
print("Number of available bikes:", station["free_bikes"])

Closest station: 09. Rigola
Distance: 6219.58278998704 meters
Number of available bikes: 5


In [6]:
max(dataset["network"]["stations"], key=lambda station: station["free_bikes"])
def distance_from_point_2(dataset, lat, lng):
    v = [ (s, distance_coords(lat, lng, s["latitude"], s["longitude"])) for s in dataset["network"]["stations"] if s["free_bikes"] > 0 ]
    return min(v, key=lambda w: w[1])

station, distance = distance_from_point_2(dataset, 45.074512, 7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")
print("Number of available bikes:", station["free_bikes"])


Closest station: 09. Rigola
Distance: 6219.58278998704 meters
Number of available bikes: 5
